Train the model

In [1]:
from src.models.Transformer import Transformer
from src.utils.CustomSchedule import CustomSchedule

import tensorflow as tf
import numpy as np
import joblib
import time

2023-09-26 16:57:57.510272: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#Load the data
SAVE_PATH = "../../../../../../data/processed/chatbot/english/dataset.tfrecord"
SHAPE = (None, 20)

dataset = tf.data.experimental.load(SAVE_PATH, element_spec=(tf.TensorSpec(shape=SHAPE, dtype=tf.int32),
                                                                    tf.TensorSpec(shape=SHAPE, dtype=tf.int32)))

Instructions for updating:
Use `tf.data.Dataset.load(...)` instead.


[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 8655443348581543048


In [3]:
dataset

<_LoadDataset element_spec=(TensorSpec(shape=(None, 20), dtype=tf.int32, name=None), TensorSpec(shape=(None, 20), dtype=tf.int32, name=None))>

In [4]:
TOKENIZER_PATH = '../../../../../../exports/chatbot/tokenizer.pkl'

tokenizer = joblib.load(TOKENIZER_PATH)

In [9]:
tokenizer

In [12]:
#Train the model

# Hyperparameters
D_MODEL = 128 # 512
NB_LAYERS = 4 # 6
FFN_UNITS = 512 # 2048
NB_PROJ = 8
DROPOUT_RATE = 0.1
VOCAB_SIZE = len(tokenizer.word_index) + 2

transformer = Transformer(vocab_size_enc=VOCAB_SIZE,
                          vocab_size_dec=VOCAB_SIZE,
                          d_model=D_MODEL,
                          nb_layers=NB_LAYERS,
                          FFN_units=FFN_UNITS,
                          nb_proj=NB_PROJ,
                          dropout_rate=DROPOUT_RATE)

In [13]:
#Custom loss function
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                            reduction="none")
def loss_function(target, pred):
    mask = tf.math.logical_not(tf.math.equal(target, 0))
    loss_ = loss_object(target, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")

In [14]:
# leaning_rate
leaning_rate = CustomSchedule(D_MODEL)

# Optimizer
optimizer = tf.keras.optimizers.Adam(leaning_rate,
                                     beta_1=0.9,
                                     beta_2=0.98,
                                     epsilon=1e-9)

In [15]:
checkpoint_path = "../../../../../../checkpoints/chatbot/english/transformer"

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Last checkpoint restored!!")

In [16]:
EPOCHS = 10
for epoch in range(EPOCHS):
    print("Beginning of the epoch {}".format(epoch+1))
    start = time.time()

    train_loss.reset_states()
    train_accuracy.reset_states()

    for (batch, (enc_inputs, targets)) in enumerate(dataset):
        dec_inputs = targets[:, :-1]
        dec_outputs_real = targets[:, 1:]
        with tf.GradientTape() as tape:
            predictions = transformer(enc_inputs, dec_inputs, True)
            loss = loss_function(dec_outputs_real, predictions)

        gradients = tape.gradient(loss, transformer.trainable_variables)
        optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

        train_loss(loss)
        train_accuracy(dec_outputs_real, predictions)

        if batch % 50 == 0:
            print("Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f}".format(
                epoch+1, batch, train_loss.result(), train_accuracy.result()))

    ckpt_save_path = ckpt_manager.save()
    print("Saving the checkpoint of epoch # {} in {}".format(epoch+1,
                                                        ckpt_save_path))
    print("Time taken for 1 epoch: {} segs\n".format(time.time() - start))

Beginning of the epoch 1


[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 8655443348581543048


Epoch 1 Batch 0 Loss 2.4058 Accuracy 0.0000


KeyboardInterrupt: 